# Khai báo thư viện

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time

# Khởi tạo và cập nhật centroids

In [ ]:
def initialize_centroids(img, k_clusters, init_type):
    if init_type == 'random':
        # Random initialization
        centroids = np.random.randint(0, 256, size=(k_clusters, len(img[0])))
        return centroids
    elif init_type == 'in_pixels':
        # Initialization from pixels in the image
        indices = np.random.choice(img.shape[0], size=k_clusters, replace=False)
        centroids = img[indices]
        return centroids
    else:
        return None

In [ ]:
def update_centroids(img, labels, old_centroids_shape):
    new_centroids = np.zeros(old_centroids_shape)
    unique_labels = np.unique(labels)
    for label in unique_labels:
        # Get the pixels belonging to the current cluster (label)
        cluster_pixels = img[labels == label]
        # Check if the cluster has any pixels assigned to it
        if len(cluster_pixels) > 0:
            # Calculate the mean of the cluster pixels along each dimension (RGB)
            cluster_mean = np.mean(cluster_pixels, axis=0)
            new_centroids[label] = cluster_mean
    return new_centroids

# Gán nhãn cho từng điểm ảnh

In [ ]:
def label_pixels(img, centroids):
    distances = np.sqrt(np.sum((img[:, None] - centroids) ** 2, axis=2))
    labels = np.argmin(distances, axis=1)
    return labels

# Kiểm tra sự hội tụ

In [ ]:
def check_convergence(old_centroids, centroids):
    # Check convergence based on a tolerance threshold
    tolerance = 1e-3
    if np.linalg.norm(centroids - old_centroids) < tolerance:
        return True
    return False

# Thuật toán k-means

In [ ]:
def kmeans(img_1d, k_clusters, max_iter, init_centroids):
    centroids = initialize_centroids(img_1d, k_clusters, init_centroids)
    labels = np.full(img_1d.shape[0], -1)

    for _ in range(max_iter):
        old_centroids = centroids.copy()

        # Assign labels to pixels based on current centroids
        labels = label_pixels(img_1d, centroids)

        # Update centroids based on current labels
        centroids = update_centroids(img_1d, labels, centroids.shape)

        # Check convergence
        if check_convergence(old_centroids, centroids):
            break

    return centroids, labels

# Tải ảnh, nén ảnh và xuất ảnh

In [ ]:
def load_image(input_file):
    while True:
        valid_formats = ['.png', '.jpg']
        if any(input_file.endswith(fmt) for fmt in valid_formats):
            try:
                # Open image
                image = Image.open(input_file)

                # Convert image to numpy array
                image = np.array(image)

                return image
            except IOError:
                print("Error: Failed to open image file.")
        else:
            print("Invalid file format. Please enter a file with .png or .jpg extension.")
            input_file = input("Enter the image filename: ")

In [ ]:
def flatten_image(image):
    # Flat image to a 1D array
    flat_image = image.reshape(image.shape[0] * image.shape[1], image.shape[2])

    return flat_image

In [ ]:
def export_image(image, k_clusters, output_format):
    # With every pixel, replace that pixel with its centroid
    result = centroids[labels].astype(np.uint8)

    # Reshape to the original shape
    result = result.reshape(image.shape)

    # Export image to file
    output_file = f'output_k{k_clusters}.{output_format}'
    if output_format == 'png':
        Image.fromarray(result).save(output_file)
    elif output_format == 'pdf':
        plt.imsave(output_file, result)
    else:
        print("Invalid output format!")

    print(f"Image exported as {output_file}")

# Hàm main để test k-means

In [ ]:
if __name__ == "__main__":
    # Init maximum iterator for stop criterion of K-means function
    max_iter = 1000

    input_file = input("Enter the image filename: ")
    image = load_image(input_file)
    flat_image = flatten_image(image)

    k_clusters = int(input("Enter the number of clusters: "))

    init_centroids = input("Enter the centroids type (random or in_pixels): ")

    output_format = input("Enter type of picture you want to save (png or pdf): ")

    
    start_time = time.time()

    # Run K-means
    centroids, labels = kmeans(flat_image, k_clusters, max_iter, init_centroids)

    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Elapsed time: {elapsed_time} seconds")

    # With every pixel, replace that pixel with its centroid
    result = centroids[labels].astype(np.uint8)

    # Reshape to the original shape
    result = result.reshape(image.shape)

    # Determine the centroid type for the title
    centroid_type = "random" if init_centroids == "random" else "in_pixels"

    # Create subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

   
    axs[0].imshow(image)
    axs[0].set_title("Original Image")

    
    axs[1].imshow(result)
    axs[1].set_title(f"K-means Result\nk_cluster={k_clusters}, Centroid Type={centroid_type}")

    plt.tight_layout()  
    plt.show()

    if output_format not in ['png', 'pdf']:
        print('Invalid format!')
    else:
        output_file = input_file.split('.')[0] + f'_k{k_clusters}_{centroid_type}.{output_format}'
        Image.fromarray(result).save(output_file)
        print(f"Image saved as {output_file}")

In [ ]:
def main():
    input_file = input("Enter the image filename: ")

    # Load the image using PIL
    try:
        img = Image.open(input_file)
    except FileNotFoundError:
        print("Error: Image file not found.")
        return
    img_copy = img.copy()

    # Save the adjusted images as PNG files
    filename, extension = input_file.split('.')

    while True:
        print("\nMenu:")
        print("0: Process all functions and save images")
        print("1: Adjust brightness and save the image")
        print("2: Adjust contrast and save the image")
        print("3: Flip the image horizontally/vertically and save it")
        print("4: Convert RGB to grayscale and sepia and save images")
        print("5: Blur and sharpen the image and save images")
        print("6: Crop the image (center) and save it")
        print("7: Crop the image in a circular region and save it")
        print("8: Crop the image in an elliptical region and save it")
        print("9: Exit")
        choice = input("Enter your choice (0-9): ")
        if choice == '0':
            # Create the first figure with 8 rows and 2 columns
            plt.figure(figsize=(10, 40))

            # Show the original image
            plt.subplot(8, 2, 1)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Adjust brightness of the image
            brightness = 50
            adjusted_img_brightness = adjust_brightness(img_copy, brightness)

            # Show the adjusted image (brightness)
            plt.subplot(8, 2, 2)
            plt.imshow(adjusted_img_brightness)
            plt.title("Brightness Adjusted Image")
            plt.axis("off")

            # Adjust contrast of the original image
            contrast = 2.0
            adjusted_img_contrast = adjust_contrast(img_copy, contrast)

            # Show the original image
            plt.subplot(8, 2, 3)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the adjusted image (contrast)
            plt.subplot(8, 2, 4)
            plt.imshow(adjusted_img_contrast)
            plt.title("Contrast Adjusted Image")
            plt.axis("off")

            # Convert the original image to grayscale
            gray_image = rgb_to_gray(np.array(img_copy))

            # Show the original image
            plt.subplot(8, 2, 5)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the grayscale image
            plt.subplot(8, 2, 6)
            plt.imshow(gray_image, cmap='gray')
            plt.title("Grayscale Image")
            plt.axis("off")

            # Convert the original image to sepia
            sepia_image = rgb_to_sepia(np.array(img_copy))

            # Show the original image
            plt.subplot(8, 2, 7)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the sepia image
            plt.subplot(8, 2, 8)
            plt.imshow(sepia_image)
            plt.title("Sepia Image")
            plt.axis("off")

            # Convert the grayscale image to a blurred grayscale image
            blurred_gray_image = blur_image(gray_image, kernel_size=5)

            # Show the grayscale image
            plt.subplot(8, 2, 9)
            plt.imshow(gray_image, cmap='gray')
            plt.title("Grayscale Image")
            plt.axis("off")

            # Show the blurred grayscale image
            plt.subplot(8, 2, 10)
            plt.imshow(blurred_gray_image, cmap='gray')
            plt.title("Blurred Grayscale Image")
            plt.axis("off")
            
            # Show the layout with the images
            plt.tight_layout()
            plt.show()

            # Sharpen the blurred grayscale image using a 5x5 kernel
            sharpened_blurred_gray_image = sharpen_image(blurred_gray_image, kernel_size=5)

            # Create the second figure with 8 rows and 2 columns
            plt.figure(figsize=(10, 40))

            # Show the blurred grayscale image
            plt.subplot(8, 2, 1)
            plt.imshow(blurred_gray_image, cmap='gray')
            plt.title("Blurred Grayscale Image")
            plt.axis("off")

            # Show the sharpened blurred grayscale image
            plt.subplot(8, 2, 2)
            plt.imshow(sharpened_blurred_gray_image, cmap='gray')
            plt.title("Sharpened Blurred Grayscale Image")
            plt.axis("off")

            # Crop the center of the image to size 200x200
            cropped_image = crop_center_image(img_copy, target_width=200, target_height=200)

            # Show the original image
            plt.subplot(8, 2, 3)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the cropped image
            plt.subplot(8, 2, 4)
            plt.imshow(cropped_image)
            plt.title("Cropped Image (200x200)")
            plt.axis("off")

            # Crop the circular region from the image
            cropped_circular_image = crop_circular_image(img_copy)

            # Show the original image
            plt.subplot(8, 2, 5)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the cropped circular image
            plt.subplot(8, 2, 6)
            plt.imshow(cropped_circular_image)
            plt.title("Cropped Circular Image")
            plt.axis("off")
            
            # Crop the elliptical region from the image
            cropped_ellipse_image = crop_ellipse(img_copy)

            # Show the original image
            plt.subplot(8, 2, 7)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the cropped elliptical image
            plt.subplot(8, 2, 8)
            plt.imshow(cropped_ellipse_image)
            plt.title("Cropped Elliptical Image")
            plt.axis("off")

            # Show the original image
            plt.subplot(8, 2, 9)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Flip the image horizontally
            flipped_horizontal_image = flip_horizontal(img)

            # Show the flipped image (horizontal)
            plt.subplot(8, 2, 10)
            plt.imshow(flipped_horizontal_image)
            plt.title("Flipped Horizontal Image")
            plt.axis("off")

            # Flip the image vertically
            flipped_vertical_image = flip_vertical(img)

            # Show the flipped image (vertical)
            plt.subplot(8, 2, 11)
            plt.imshow(flipped_vertical_image)
            plt.title("Flipped Vertical Image")
            plt.axis("off")

            # Show the layout with the images
            plt.tight_layout()
            plt.show()

            # Save the adjusted images as files

            output_filename_brightness = f"{filename}_brightness_{int(brightness)}.{extension}"
            output_filename_contrast = f"{filename}_contrast_{int(contrast * 100)}.{extension}"
            output_filename_gray = f"{filename}_gray.{extension}"
            output_filename_blurred_gray = f"{filename}_blurred_gray.{extension}"
            output_filename_sharpened_blurred_gray = f"{filename}_sharpened_blurred_gray.{extension}"
            output_filename_sepia = f"{filename}_sepia.{extension}"
            output_filename_cropped = f"{filename}_cropped_200x200.{extension}"
            output_filename_flipped_horizontal = f"{filename}_flipped_horizontal.{extension}"
            output_filename_flipped_vertical = f"{filename}_flipped_vertical.{extension}"
            output_filename_cropped_circular = f"{filename}_cropped_circular.{extension}"
            output_filename_cropped_elipse = f"{filename}_cropped_elipse.{extension}"
            

            plt.imsave(output_filename_brightness, np.array(adjusted_img_brightness))
            plt.imsave(output_filename_contrast, np.array(adjusted_img_contrast))
            plt.imsave(output_filename_gray, gray_image, cmap='gray')
            plt.imsave(output_filename_blurred_gray, blurred_gray_image, cmap='gray')
            plt.imsave(output_filename_sharpened_blurred_gray, sharpened_blurred_gray_image, cmap='gray')
            plt.imsave(output_filename_sepia, sepia_image)
            plt.imsave(output_filename_cropped, np.array(cropped_image))
            plt.imsave(output_filename_flipped_horizontal, np.array(flipped_horizontal_image))
            plt.imsave(output_filename_flipped_vertical, np.array(flipped_vertical_image))
            plt.imsave(output_filename_cropped_circular, cropped_circular_image)
            plt.imsave(output_filename_cropped_elipse, cropped_ellipse_image)

        elif choice == '1':
            # Show the original image
            plt.subplot(1, 2, 1)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Adjust brightness of the image
            brightness = 50
            adjusted_img_brightness = adjust_brightness(img_copy, brightness)

            # Show the adjusted image (brightness)
            plt.subplot(1, 2, 2)
            plt.imshow(adjusted_img_brightness)
            plt.title("Brightness Adjusted Image")
            plt.axis("off")

            # Show the layout with the images
            plt.tight_layout()
            plt.show()

            output_filename_brightness = f"{filename}_brightness_{int(brightness)}.{extension}"

            plt.imsave(output_filename_brightness, np.array(adjusted_img_brightness))

        elif choice == '2':
            # Adjust contrast of the original image
            contrast = 2.0
            adjusted_img_contrast = adjust_contrast(img_copy, contrast)

            # Show the original image
            plt.subplot(1, 2, 1)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the adjusted image (contrast)
            plt.subplot(1, 2, 2)
            plt.imshow(adjusted_img_contrast)
            plt.title("Contrast Adjusted Image")
            plt.axis("off")

             # Show the layout with the images
            plt.tight_layout()
            plt.show()

            output_filename_contrast = f"{filename}_contrast_{int(contrast * 100)}.{extension}"
            plt.imsave(output_filename_contrast, np.array(adjusted_img_contrast))

        elif choice == '3':
            # Flip the image horizontally
            flipped_horizontal_image = flip_horizontal(img)

            # Show the flipped image (horizontal)
            plt.subplot(8, 2, 10)
            plt.imshow(flipped_horizontal_image)
            plt.title("Flipped Horizontal Image")
            plt.axis("off")

            # Flip the image vertically
            flipped_vertical_image = flip_vertical(img)

            # Show the flipped image (vertical)
            plt.subplot(8, 2, 11)
            plt.imshow(flipped_vertical_image)
            plt.title("Flipped Vertical Image")
            plt.axis("off")

            # Show the layout with the images
            plt.tight_layout()
            plt.show()

            output_filename_flipped_horizontal = f"{filename}_flipped_horizontal.{extension}"
            output_filename_flipped_vertical = f"{filename}_flipped_vertical.{extension}"

            plt.imsave(output_filename_flipped_horizontal, np.array(flipped_horizontal_image))
            plt.imsave(output_filename_flipped_vertical, np.array(flipped_vertical_image))

        elif choice == '4':
             # Convert the original image to grayscale
            gray_image = rgb_to_gray(np.array(img_copy))

            # Show the original image
            plt.subplot(1, 3, 1)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the grayscale image
            plt.subplot(1, 3, 2)
            plt.imshow(gray_image, cmap='gray')
            plt.title("Grayscale Image")
            plt.axis("off")

            # Convert the original image to sepia
            sepia_image = rgb_to_sepia(np.array(img_copy))

            # Show the original image
            plt.subplot(1, 3, 3)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the sepia image
            plt.figure(figsize=(10, 5))  # Create a new figure to display only the sepia image
            plt.imshow(sepia_image)
            plt.title("Sepia Image")
            plt.axis("off")

             # Show the layout with the images
            plt.tight_layout()
            plt.show()

            # Save the images
            filename, extension = input_file.split('.')
            output_filename_gray = f"{filename}_gray.{extension}"
            output_filename_sepia = f"{filename}_sepia.{extension}"

            plt.imsave(output_filename_gray, gray_image, cmap='gray')
            plt.imsave(output_filename_sepia, sepia_image)

        elif choice=='5':
            # Convert the original image to grayscale
            gray_image = rgb_to_gray(np.array(img_copy))

            # Convert the grayscale image to a blurred grayscale image
            blurred_gray_image = blur_image(gray_image, kernel_size=5)

            # Sharpen the blurred grayscale image using a 5x5 kernel
            sharpened_blurred_gray_image = sharpen_image(blurred_gray_image, kernel_size=5)

            # Show the grayscale image
            plt.subplot(1, 3, 1)
            plt.imshow(gray_image, cmap='gray')
            plt.title("Grayscale Image")
            plt.axis("off")

            # Show the blurred grayscale image
            plt.subplot(1, 3, 2)
            plt.imshow(blurred_gray_image, cmap='gray')
            plt.title("Blurred Grayscale Image")
            plt.axis("off")
            
            # Show the sharpened blurred grayscale image
            plt.subplot(1, 3, 3)
            plt.imshow(sharpened_blurred_gray_image, cmap='gray')
            plt.title("Sharpened Blurred Grayscale Image")
            plt.axis("off")

            # Show the layout with the images
            plt.tight_layout()
            plt.show()

            output_filename_gray = f"{filename}_gray.{extension}"
            output_filename_blurred_gray = f"{filename}_blurred_gray.{extension}"
            output_filename_sharpened_blurred_gray = f"{filename}_sharpened_blurred_gray.{extension}"

            plt.imsave(output_filename_gray, gray_image, cmap='gray')
            plt.imsave(output_filename_blurred_gray, blurred_gray_image, cmap='gray')
            plt.imsave(output_filename_sharpened_blurred_gray, sharpened_blurred_gray_image, cmap='gray')

        elif choice == '6':
            # Crop the center of the image to size 200x200
            cropped_image = crop_center_image(img_copy, target_width=200, target_height=200)

            # Show the original image
            plt.subplot(1, 2, 1)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the cropped image
            plt.subplot(1, 2, 2)
            plt.imshow(cropped_image)
            plt.title("Cropped Image (200x200)")
            plt.axis("off")

             # Show the layout with the images
            plt.tight_layout()
            plt.show()

            output_filename_cropped = f"{filename}_cropped_200x200.{extension}"

        elif choice =='7':
            # Crop the circular region from the image
            cropped_circular_image = crop_circular_image(img_copy)

            # Show the original image
            plt.subplot(1, 2, 1)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the cropped circular image
            plt.subplot(1, 2, 2)
            plt.imshow(cropped_circular_image)
            plt.title("Cropped Circular Image")
            plt.axis("off")

            # Show the layout with the images
            plt.tight_layout()
            plt.show()

            output_filename_cropped_circular = f"{filename}_cropped_circular.{extension}"
            plt.imsave(output_filename_cropped_circular, cropped_circular_image)
            
        
        elif choice =='8':
            # Crop the elliptical region from the image
            cropped_ellipse_image = crop_ellipse(img_copy)

            # Show the original image
            plt.subplot(1, 2, 1)
            plt.imshow(img)
            plt.title("Original Image")
            plt.axis("off")

            # Show the cropped elliptical image
            plt.subplot(1, 2, 2)
            plt.imshow(cropped_ellipse_image)
            plt.title("Cropped Elliptical Image")
            plt.axis("off")

            # Show the layout with the images
            plt.tight_layout()
            plt.show()

            output_filename_cropped_elipse = f"{filename}_cropped_elipse.{extension}"
            plt.imsave(output_filename_cropped_elipse, cropped_ellipse_image)

        elif choice == '9':
            break

        else:
            print("Invalid choice. Please enter a valid option (0-9).")




    

if __name__ == "__main__":
    main()
